In [1]:
import pandas as pd
import numpy as np
import os
import time
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno

os.chdir('/Users/Clair/Desktop/FYP/data')

In [ ]:
#1: import all sample dataset
main_abs = pd.read_csv("main_abs.csv")
main_log = pd.read_csv("main_log.csv")
main_yoy = pd.read_csv("main_yoy.csv")
main_qoq = pd.read_csv("main_qoq.csv")
main_dep = pd.read_csv("main_dependent.csv")

In [2]:
#2: import feature selection dataset -> dictionary separate feature by needed formats (type_dict)
bytype = pd.read_csv('pca1_select_type.csv',header = None)
ie = pd.read_csv('ie_70.csv')
bytype = pd.merge(bytype, ie, left_on = 0, right_on = ['long'], how = 'left')

grouped = bytype.groupby(1)
type_dict = {}
for name, g in grouped:
    type_dict[name] = g['short'].dropna().to_list()

num_col = list(set(bytype['short'].dropna())) # all feature columns

label_col = ['gvkey', 'datacqtr', 'datafqtr', 'gsector', 'cquarter',
                 'cyear', 'cyeargvkey', 'gvkeydatafqtr']

In [63]:
# 3: select from each individual database and concat together -> dataframe in needed format (main)
main_label = main_abs.filter(label_col)
main_abs_select = main_abs.filter(type_dict['Absolute'])
main_log_select = main_log.filter(type_dict['Log'])
main_qoq_select = main_qoq.filter(type_dict['QoQ'])
main_yoy_select = main_yoy.filter(type_dict['YoY'])
main = pd.concat([main_label, main_abs_select, main_log_select,
                  main_qoq_select, main_yoy_select], axis=1)

In [3]:
'''starting point 1 - main.shape = (309036, 78)'''
main = pd.read_csv('main_selected.csv')

In [4]:
#3.1: fillna
'''
1. fill YoY, QoQ -> 0
2. fill abs, log -> last observation
3. fill abs, log -> next observation
4. fill rest -> 0
'''
for i in ['YoY', 'QoQ']:
    main[type_dict[i]] = main[type_dict[i]].fillna(0)
for i in ['Absolute', 'Log']:
    main[type_dict[i]] = main.groupby('gvkey').apply(lambda x: x.fillna(method = 'ffill'))[type_dict[i]]
    main[type_dict[i]] = main.groupby('gvkey').apply(lambda x: x.fillna(method = 'bfill'))[type_dict[i]]
main = main.fillna(0)

In [5]:
#3.2: high correlation detection
ex_corr_col = ['epspxq'] + label_col

def drop_dup_nonsequal(df, col1, col2):
    mask = df[col1] < df[col2]
    df['first'] = df[col1].where(mask, df[col2])
    df['second'] = df[col2].where(mask, df[col1])
    df = df.drop_duplicates(subset=['first', 'second'])
    del df['first']
    del df['second']
    return df

def high_corr(df, threshold = 0.9):
    corr = df.corr().abs()
    s = corr.unstack()
    so = s.sort_values(kind="quicksort",ascending=False).to_frame().reset_index()
    so.columns = ['v1','v2','corr']
    so = so.loc[(so['v1']!=so['v2']) & (so['corr'] > threshold)].drop_duplicates(subset = ['v1','v2'])
    so_drop = drop_dup_nonsequal(so, 'v1', 'v2')
    return so_drop

def del_correlation(df, threshold = 0.9, ex_corr_col = ex_corr_col):
        col_corr = set() # Set of all the names of deleted columns
        corr_matrix = df.corr()
        high_corr_df = high_corr(df)
        for i in range(len(corr_matrix.columns)):
            for j in range(i):
                if (corr_matrix.iloc[i, j] >= threshold) and (corr_matrix.columns[j] not in col_corr):
                    colname = corr_matrix.columns[i] # getting the name of column
                    col_corr.add(colname)
        del_corr = col_corr - set(ex_corr_col)
        del_value = high_corr_df.loc[(high_corr_df['v1'].isin(del_corr))|(high_corr_df['v2'].isin(del_corr))]
        del_col_ie = ie.loc[ie['short'].isin(del_corr)]
        return del_corr, del_col_ie, del_value 

del_corr, del_col_ie, del_value = del_correlation(main)

In [6]:
#3.3: review and decide to be deleted high correlation items
del_corr_s = set(['chq', 'dvpsxq', 'epsfxq', 'epspiq', 'lseq', 'miiq',
              'oepsxq', 'opepsq', 'rcpq', 'stkcoq', 'wdpq', 'xidoq'])
main = main.filter(set(main.columns) - del_corr_s)
num_col = list(set(main.columns) - set(label_col))

In [6]:
# 4: construct PCA dataframe -> whole dataframe (df_pca) for 1980 -2020
gvkey = list(set(main['gvkey']))        # gvkey list = 4592
datacqtr = list(set(main['datacqtr']))  # cqtr list = 158 (1980Q2 - 2020Q1)

df_pca = pd.DataFrame()
df_pca['gvkey'] = gvkey

grouped = main.groupby('datacqtr')
for cqtr, g in grouped:
    g_num = g.filter(num_col + ['gvkey'])
    g_num.columns = [col + '_' + cqtr for col in num_col] + ['gvkey']
    df_pca = pd.merge(df_pca, g_num, on = ['gvkey'], how = 'left')

In [7]:
'''starting point 2 - df_pca.shape = (4592, 11271)'''
df_pca.to_csv('all_pca_full.csv')
#df_pca = pd.read_csv('df_pca_full.csv')

In [8]:
# 5: extract dataframe for selected window[3, 5] -> dataframe (pca_df_5) + (pca_df_5)
def filter_by_window(window, last_year = 2019, df_pca = df_pca):
    cqtr_window =[]
    for yr in np.arange(last_year - window,last_year,1):
        cqtr_window.extend([str(yr)+'Q'+str(i+1) for i in range(4)]) # list out needed timing (e.g. 2018Q4)

    num_col_cqtr = []
    for t in cqtr_window:
        num_col_cqtr.extend([col + '_' + t for col in num_col])
    print(cqtr_window[0], cqtr_window[-1], len(num_col_cqtr))
    
    df_pca_sub = df_pca.filter(['gvkey'] + num_col_cqtr)
    df_pca_sub = df_pca_sub.dropna(subset = num_col_cqtr)
    df_pca_sub.to_csv('all_pca_'+str(window)+'.csv', index = False)
    return df_pca_sub

df_pca_3 = filter_by_window(3)
df_pca_5 = filter_by_window(5)

2016Q1 2018Q4 840
2014Q1 2018Q4 1400


# Working (not useful)

In [ ]:
from collections import Counter
lst = k['v1'].to_list() + k['v2'].to_list()
dict1 = Counter(lst)
count_df = pd.DataFrame(list(dict1.items()), columns=[
                        'var', 'count']).sort_values(by=['count'], ascending=False)
k['count_v1'] = k.v1.map(count_df.set_index('var')['count'])
k['count_v2'] = k.v2.map(count_df.set_index('var')['count'])
del_corr = set()
condition = k['count_v1'] > k['count_v2']
del_lst = k.loc[condition, 'v1'].to_list() + k.loc[~condition, 'v2'].to_list()

In [48]:
def correlation(high_corr_df, threshold):
    col_corr = set() # Set of all the names of deleted columns
    for i in count_lst:
        if (high_corr_df.loc[high_corr_df['v1']==i,'v2'] not in col_corr):
            colname = corr_matrix.columns[i] # getting the name of column
                col_corr.add(colname)
                if colname in dataset.columns:
                    del dataset[colname] # deleting the column from the dataset
    return dataset
correlation(k, 0.9)

1 0
2 0
2 1
3 0
3 1
3 2
4 0
4 1
4 2
4 3
5 0
5 1
5 2
5 3
5 4
6 0
6 1
6 2
6 3
6 4
6 5
7 0
7 1
7 2
7 3
7 4
7 5
7 6
8 0
8 1
8 2
8 3
8 4
8 5
8 6
8 7
9 0
9 1
9 2
9 3
9 4
9 5
9 6
9 7
9 8
10 0
10 1
10 2
10 3
10 4
10 5
10 6
10 7
10 8
10 9
11 0
11 1
11 2
11 3
11 4
11 5
11 6
11 7
11 8
11 9
11 10
12 0
12 1
12 2
12 3
12 4
12 5
12 6
12 7
12 8
12 9
12 10
12 11
13 0
13 1
13 2
13 3
13 4
13 5
13 6
13 7
13 8
13 9
13 10
13 11
13 12
14 0
14 1
14 2
14 3
14 4
14 5
14 6
14 7
14 8
14 9
14 10
14 11
14 12
14 13
15 0
15 1
15 2
15 3
15 4
15 5
15 6
15 7
15 8
15 9
15 10
15 11
15 12
15 13
15 14
16 0
16 1
16 2
16 3
16 4
16 5
16 6
16 7
16 8
16 9
16 10
16 11
16 12
16 13
16 14
16 15
17 0
17 1
17 2
17 3
17 4
17 5
17 6
17 7
17 8
17 9
17 10
17 11
17 12
17 13
17 14
17 15
17 16
18 0
18 1
18 2
18 3
18 4
18 5
18 6
18 7
18 8
18 9
18 10
18 11
18 12
18 13
18 14
18 15
18 16
18 17
19 0
19 1
19 2
19 3
19 4
19 5
19 6
19 7
19 8
19 9
19 10
19 11
19 12
19 13
19 14
19 15
19 16
19 17
19 18
20 0
20 1
20 2
20 3
20 4
20 5
20 6
20 7
20 8
20 9


,gvkey,datacqtr,datafqtr,gsector,cquarter,cyear,gvkeydatafqtr,epsfiq,actq,aocipenq,...,dpq,mibtq,ppentq,pstkq,teqq,tstkq,txdbq,txtq,xaccq,xintq
0,1034,1983Q4,1983Q4,35.0,4,1983,10341983Q4,0.22,0.310926,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1034,1984Q1,1984Q1,35.0,1,1984,10341984Q1,0.28,0.305161,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1034,1984Q2,1984Q2,35.0,2,1984,10341984Q2,0.30,0.306713,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1034,1984Q3,1984Q3,35.0,3,1984,10341984Q3,0.28,0.310589,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1034,1984Q4,1984Q4,35.0,4,1984,10341984Q4,0.26,0.290082,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1034,1985Q1,1985Q1,35.0,1,1985,10341985Q1,0.33,0.313539,NaN,...,NaN,NaN,0.815010,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1034,1985Q2,1985Q2,35.0,2,1985,10341985Q2,0.36,0.315060,NaN,...,NaN,NaN,-1.230237,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1034,1985Q3,1985Q3,35.0,3,1985,10341985Q3,0.23,0.324095,NaN,...,NaN,NaN,-2.960457,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1034,1985Q4,1985Q4,35.0,4,1985,10341985Q4,0.21,0.329145,NaN,...,NaN,NaN,-1.584406,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1034,1986Q1,1986Q1,35.0,1,1986,10341986Q1,0.24,0.310985,NaN,...,NaN,NaN,4.486842,NaN,NaN,NaN,NaN,NaN,NaN,NaN
